# Toronto Project (part 1 of 3)

*Stefano Magarotto*

This first part consists in exploring, segmenting, and clustering the neighborhoods in the city of Toronto.
The data is available in a Wikipedia page. I will get, wrangle and clean the retrieved data.
Finally I will put it a pandas dataframe so that it is in a structured format.

### Step 1. Import libraries 

In [1]:
# First we import all libraries we will need
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Step 2. Read the wikipedia page and convert in a dataframe

In [21]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Step 3. Wrangle and clean data

In [24]:
# Fix column names
df.rename(columns={"Postcode": "PostalCode", "Neighbourhood":"Neighborhood"}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [25]:
# Remove Boroughs that are 'Not assigned'
df = df[df['Borough'] != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [26]:
# More than one neighborhood can exist in one postal code area 
# so I combine these into one row with the neighborhoods separated with a comma
df["Neighborhood"] = df.groupby("PostalCode")["Neighborhood"].transform(lambda x: ', '.join(x))

# Remove duplicates
df = df.drop_duplicates()

# Update index to be PostalCode 
if(df.index.name != 'PostalCode'):
    df = df.set_index('PostalCode')
    
df.head()

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


In [27]:
# If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough
df['Neighborhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


### Step 4. Requested output

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [28]:
df.shape

(103, 2)